<h1> LinkedIn Scrapping </h1>

<h3> Librerias y requerimientos. </h3>

- Instalación de selenium y beautifulsoup4

In [38]:
#!pip install -r requirements.txt

- Instalación de driver para uso de selenium en Google Chrome.

1. Ingresar a: https://sites.google.com/chromium.org/driver/
2. Ingresar a Downloads.
3. Seleccionar la versión del driver de acuerdo a la versión de Google Chrome y el sistema operativo donde será instalado.
4. Descomprimir el archivo descargado y abrir el archivo ejecutable.
5. Extraer el path o ruta donde se encuenta el archivo chromedriver.exe

- Importar librerias.

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
import pandas as pd
import warnings
from datetime import date
warnings.filterwarnings("ignore")

In [2]:
fecha = date.today().strftime('%Y-%m-%d')

<h3> 1. LinkedIn </h3>

In [4]:
#Definir el path del chromedriver.exe
ChromePath = "C:\chromedriver.exe"

#Credenciales LinkedIn
email = ""
pwd = ""

#Busqueda
search = input("- ¿Qué desea buscar?: ")
results = int(input("- ¿Cuántas páginas desea consultar?: "))

- ¿Qué desea buscar?: developer sr mexico
- ¿Cuántas páginas desea consultar?: 10


In [5]:
#Web Driver Object
driver = webdriver.Chrome(executable_path=ChromePath)
driver.maximize_window()
driver.get("https://www.linkedin.com/login")

time.sleep(3) #Tiempo de espera para cargar la página

#Login a LinkeIn
driver.find_element_by_id('username').send_keys(email)
driver.find_element_by_id('password').send_keys(pwd)
driver.find_element_by_id('password').send_keys(Keys.RETURN)

time.sleep(3)

#Ingresa busqueda en contenedor
driver.find_element_by_class_name('search-global-typeahead__input').send_keys(search)
driver.find_element_by_class_name('search-global-typeahead__input').send_keys(Keys.RETURN)

time.sleep(3)

#Expandir resultados de personas
driver.find_element_by_link_text("Ver todos los resultados de personas").click()

time.sleep(1)
lista_maestra = []
for i in range(1,results+1):
    time.sleep(1)
    if i == 1:
        profiles = driver.find_elements_by_xpath("//a[@class='app-aware-link']") #Extraer URL de todos los perfiles
        time.sleep(2)
        lista_url = []
        for j in profiles:
            var = j.get_attribute('href')
            if ("SHARED_CONNECTIONS_CANNED_SEARCH" in var) or ("/in/ACo" in var): #Tomar perfiles solo del listado
                continue
            else:
                lista_url.append(var) #Extraer URL
        print("- Página 1: Cargada.")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
    else: #Control de paginación
        try:
            driver.find_element_by_xpath(f'//li[@data-test-pagination-page-btn="{i}"]').click()
            time.sleep(2)
            profiles = driver.find_elements_by_xpath("//a[@class='app-aware-link']")
            time.sleep(1)
            lista_url = []
            for j in profiles:
                var = j.get_attribute('href')
                if ("SHARED_CONNECTIONS_CANNED_SEARCH" in var) or ("/in/ACo" in var):
                    continue
                else:
                    lista_url.append(var)
            print(f"- Página {i}: Cargada.")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
        except:
            driver.find_element_by_xpath(f'//button[@aria-label="Página {i}"]').click()
            time.sleep(2)
            profiles = driver.find_elements_by_xpath("//a[@class='app-aware-link']")
            time.sleep(1)
            lista_url = []
            for j in profiles:
                var = j.get_attribute('href')
                if ("SHARED_CONNECTIONS_CANNED_SEARCH" in var) or ("/in/ACo" in var):
                    continue
                else:
                    lista_url.append(var)
            print(f"- Página {i}: Cargada.")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    
    lista_maestra = lista_maestra + lista_url #Listado maestro de URL

- Página 1: Cargada.
- Página 2: Cargada.
- Página 3: Cargada.
- Página 4: Cargada.
- Página 5: Cargada.
- Página 6: Cargada.
- Página 7: Cargada.
- Página 8: Cargada.
- Página 9: Cargada.
- Página 10: Cargada.


In [6]:
lista_maestra = [ x for x in lista_maestra if "https://www.linkedin.com/search/results/people/headless?origin=CLUSTER_EXPANSION&keywords=" not in x ]

In [7]:
df = pd.DataFrame(columns = ["Nombre","Rol","Empresa","Ubicacion","LinkedIn URL"]) #Estructura DataFrame final
i = 1
for url in lista_maestra:
    driver.get(url)
    intro = BeautifulSoup(driver.page_source, 'lxml').find('div', {'class': 'mt2 relative'}) #Convertir página a xml
    
    nombre = intro.find("h1").get_text().strip()
    Rol = intro.find("div", {'class': 'text-body-medium'}).get_text().strip()
    
    try:
        empresa = intro.find("li", {'class': 'pv-text-details__right-panel-item'}).get_text().strip()
    except:
        empresa = ""
    
    try:
        ubicacion = intro.find("span", {'class': 'text-body-small inline t-black--light break-words'}).get_text().strip()
    except:
        ubicacion = ""
        
    row = pd.Series([nombre,Rol,empresa,ubicacion,url], index = df.columns)
    
    #Población de DataFrame con datos de cada perfil
    df = df.append(row, ignore_index=True)
    time.sleep(1)

df.head()

,Nombre,Rol,Empresa,Ubicacion,LinkedIn URL
0,Luis Eduardo Navarrete Ramos,Sr. backend developer (Python),Terminal,"Benito Juárez, Ciudad de México, México",https://www.linkedin.com/in/luis-eduardo-navar...
1,Isabel Sánchez,Senior Data Systems Enginner(Bussines Intellig...,,"Naucalpan de Juárez, México, México",https://www.linkedin.com/in/isabel-s%C3%A1nche...
2,Christian Andres Lopez Valdez,Sr Python Developer @ TrueLogic,,"Toluca, México, México",https://www.linkedin.com/in/catfor?miniProfile...
3,Cristian Alejandro Hernández,Data Engineer @ Coppel,,"Puebla, Puebla, México",https://www.linkedin.com/in/cristian-alejandro...
4,Juan Felipe Bernal,Business Developer Manager - México / Elenas,Elenas,"Bogotá, Distrito Capital, Colombia",https://www.linkedin.com/in/juanbernal92?miniP...


In [8]:
#Exportar Datos
df.to_excel(f"{fecha} LinkedIn Scrapping - {search}.xlsx")
print(f"- Se exportaron {len(df)} perfiles.")

- Se exportaron 83 perfiles.
